In [1]:
!python run.py

Interface starting at http://127.0.0.1:8521


Traceback (most recent call last):
  File "C:\Users\PrakrutUpadhyay\Downloads\Engg-3130-Project\run.py", line 3, in <module>
    server.launch()
  File "C:\Users\PrakrutUpadhyay\anaconda3\lib\site-packages\mesa\visualization\ModularVisualization.py", line 403, in launch
    self.listen(self.port)
  File "C:\Users\PrakrutUpadhyay\anaconda3\lib\site-packages\tornado\web.py", line 2109, in listen
    server.listen(port, address)
  File "C:\Users\PrakrutUpadhyay\anaconda3\lib\site-packages\tornado\tcpserver.py", line 151, in listen
    sockets = bind_sockets(port, address=address)
  File "C:\Users\PrakrutUpadhyay\anaconda3\lib\site-packages\tornado\netutil.py", line 161, in bind_sockets
    sock.bind(sockaddr)
OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted
